# 3. Heat-Pipe Design [10 pts]

A heated fluid at temperature $T$ (degrees above ambient temperature) flows in a pipe with fixed length and circular cross section with radius $r$. A layer of insulation, with thickness $w$, surrounds the pipe to reduce heat loss through the pipe walls ($w$ is much smaller than $r$). The design variables in this problem are $T$, $r$, and $w$.

The energy cost due to heat loss is roughly equal to $α_1Tr/w$. The cost of the pipe, which has a fixed wall thickness, is approximately proportional to the total material, i.e., it is given by $α_2r$. The cost of the insulation is also approximately proportional to the total insulation material, i.e., roughly $α_3rw$. The total cost is the sum of these three costs.

The heat flow down the pipe is entirely due to the flow of the fluid, which has a fixed velocity, i.e., it is given by $α_4Tr^2$. The constants $α_i$ with $i ∈ \{1,2,3,4\}$ are __all positive__, as are the variables $T$, $r$, and $w$.

Now the problem: maximize the total heat flow down the pipe, subject to an upper limit $C_{max}$ on total cost, and the constraints

<img src = "3a.png">

__a)__ Express this problem as a geometric program, and convert it into a convex optimization problem.

<img src = "3a.jpg">

__b)__ Consider a simple instance of this problem, where $C_{max} = 500$ and $α_1 = α_2 = α_3 = α_4 = 1$. Also assume for simplicity that __each variable has a lower bound of zero__ and __no upper bound__. Solve this problem using JuMP. Use the Mosek solver and the command __@NLconstraint(...)__ to specify nonlinear constraints such as log-sum-exp functions. Note: Mosek can solve general convex optimization problems! What is the optimal $T$, $r$, and $w$?

In [1]:
using JuMP, Mosek

a = ones(4)
C_max = 500

m = Model(solver=MosekSolver(LOG=0))

@variable(m, x >= 0)
@variable(m, y >= 0)
@variable(m, z >= 0)

@NLexpression(m, T, exp(x))
@NLexpression(m, r, exp(y))
@NLexpression(m, w, exp(z))

@NLexpression(m, heat_loss_cost, a[1]*T*r/w)
@NLexpression(m, pipe_cost, a[2]*r)
@NLexpression(m, insulation_cost, a[3]*r*w)
@NLexpression(m, total_cost, heat_loss_cost + pipe_cost + insulation_cost)

@NLconstraint(m, exp(log(a[1]) + x + y - z) + exp(log(a[2]) + y) + exp(log(a[3]) + y + z) - C_max <= 0)
@NLconstraint(m, exp(z - y) <= 0.1)

@NLobjective(m, Min, log(a[4]) - x - 2y)

println(solve(m))

println("We achieved a maximum heat flow of ", getvalue(T) * getvalue(r)^2)
println("T = ", getvalue(T))
println("r =  ", getvalue(r))
println("w = ", getvalue(w))
println("Our total cost is: ", getvalue(total_cost))

Optimal
We achieved a maximum heat flow of 51305.90644653668
T = 23.840238958644314
r =  46.39042810824172
w = 4.639042747423223
Our total cost is: 500.0000000182839


In [2]:
# attempting something I saw on Piazza (@196)
# inverted objective, then solve for minimization
# NOT MY FINAL ANSWER

using JuMP, Mosek

m1 = Model(solver=MosekSolver(LOG=0))

@variable(m1, T >= 0)
@variable(m1, r >= 0)
@variable(m1, w >= 0)

@NLexpression(m1, x, log(T))
@NLexpression(m1, y, log(r))
@NLexpression(m1, z, log(w))

@NLexpression(m1, heat_loss_cost, a[1]*T*r/w)
@NLexpression(m1, pipe_cost, a[2]*r)
@NLexpression(m1, insulation_cost, a[3]*r*w)
@NLexpression(m1, total_cost, heat_loss_cost + pipe_cost + insulation_cost)

@NLconstraint(m1, a[1]*T*r/w + a[2]*r + a[3]*r*w <= C_max)
@NLconstraint(m1, w <= 0.1*r)

@NLobjective(m1, Min, (a[4]*T*r^2)^-1)

println(solve(m1))

println("We achieved a maximum heat flow of ", getvalue(T) * getvalue(r)^2)
println("T = ", getvalue(T))
println("r =  ", getvalue(r))
println("w = ", getvalue(w))
println("Our total cost is: ", getvalue(total_cost))

Optimal
We achieved a maximum heat flow of 48667.67592771966
T = 20.24756171792293
r =  49.02684407505225
w = 4.3118627352713395
Our total cost is: 490.64317317489616
